In [20]:
import sys
sys.path.insert(0, '../../')
from src.questions_construction.questions import *
from src.questions_construction.domains import DOMAIN_NAMES
from src.analysis.model_performances import *
from src.common import *
import random
from collections import defaultdict
from copy import deepcopy

In [22]:
# data_name = 'data_for_test_zero_shot.pruned'
# ids = open_jsonl(f'{DATA_PATH}/dataset_ids.test.pruned.jsonl')
# data_name = 'questions.composite.test'
ids = open_jsonl(f'{DATA_PATH}/questions.composite.test_ids.jsonl')

In [23]:
len(set(ids))

360

In [28]:
data_name = 'questions.composite.train'

subs = [WITH_RANDOM_SUB, WITHOUT_RANDOM_SUB]
# prompt_types = ['few_shot_0']
total_instances = 0
ids_by_prefix = {}
for sub in subs:
    # for ram in RAMIFICATION_TYPES:
        # for few_shot in prompt_types:
    for domain in DOMAIN_NAMES:
        for i in range(1,11):
            try:
                # path = os.path.join(DATA_PATH, data_name, sub, ram, few_shot, domain, f'Instance_{i}.jsonl')
                path = os.path.join(DATA_PATH, data_name, sub, domain, f'Instance_{i}.jsonl')
                
                if not os.path.exists(path):
                    print(path)
                    continue
                else:
                    new_data = []
                    for d in open_jsonl(path):
                        if d['id'] not in ids:
                            new_data.append(d)
                    save_jsonl(new_data, path)
                    total_instances+= len(new_data)
            except Exception as e:
                print(e)
                print(path)
        # print(ram, few_shot, domain)
        print(domain, sub)
        

blocksworld with_random_sub
depots with_random_sub
driverlog with_random_sub
goldminer with_random_sub
grippers with_random_sub
logistics with_random_sub
miconic with_random_sub
mystery with_random_sub
npuzzle with_random_sub
satellite with_random_sub
spanner with_random_sub
visitall with_random_sub
zenotravel with_random_sub
blocksworld without_random_sub
depots without_random_sub
driverlog without_random_sub
goldminer without_random_sub
grippers without_random_sub
logistics without_random_sub
miconic without_random_sub
mystery without_random_sub
npuzzle without_random_sub
satellite without_random_sub
spanner without_random_sub
visitall without_random_sub
zenotravel without_random_sub


In [9]:
4*len(ids) - total_instances

-220

In [29]:
total_instances

29678

# Prune More

In [4]:
ids_file_name = 'dataset_ids.test'
selected_ids = open_jsonl(f'{DATA_PATH}/{ids_file_name}.jsonl')
questions_dir = f'{DATA_PATH}/questions_m1'
questions_by_id = gather_questions(questions_dir, selected_ids=selected_ids)

questions gathered


In [62]:
len(questions_by_id)

7077

In [63]:
ids_to_remove = []

## Remove FA for fluents

In [64]:
selected_data = []
for q_id, by_sub in questions_by_id.items():
    for sub, d in by_sub.items():
        if sub ==WITH_RANDOM_SUB:
            continue
        if d[OUT_OBJ_ANSWER_TYPE] == FREE_ANSWER_TYPE and d[OUT_OBJ_QUESTION_CATEGORY] == 'fluent_tracking':
            selected_data.append(d)
print(len(selected_data))
random.shuffle(selected_data)
ids_to_remove.extend([d['id'] for d in selected_data[130:]])

952


## Remove FA for Halluciantion

In [65]:
selected_data = []
for q_id, by_sub in questions_by_id.items():
    for sub, d in by_sub.items():
        if sub ==WITH_RANDOM_SUB:
            continue
        if d[OUT_OBJ_ANSWER_TYPE] == FREE_ANSWER_TYPE and d[OUT_OBJ_QUESTION_CATEGORY] == 'hallucination':
            selected_data.append(d)
print(len(selected_data))
random.shuffle(selected_data)
ids_to_remove.extend([d['id'] for d in selected_data[130:]])

240


In [66]:
len(ids_to_remove)

932

## Fluent Tracking TF

In [67]:
selected_data = []
for q_id, by_sub in questions_by_id.items():
    for sub, d in by_sub.items():
        if sub ==WITH_RANDOM_SUB:
            continue
        if d[OUT_OBJ_ANSWER_TYPE] == TRUE_FALSE_ANSWER_TYPE and d[OUT_OBJ_QUESTION_CATEGORY] == 'fluent_tracking':
            selected_data.append(d)
print(len(selected_data))
random.shuffle(selected_data)
ids_to_remove.extend([d['id'] for d in selected_data[1600:]])

1889


In [68]:
len(ids_to_remove)

1221

In [69]:
len(questions_by_id) - len(ids_to_remove)

5856

In [70]:
new_ids = [id for id in selected_ids if id not in ids_to_remove]

In [71]:
len(new_ids)

5866

In [72]:
save_jsonl(new_ids, f'{DATA_PATH}/{ids_file_name}_pruned.jsonl')